In [20]:
# This is a trick taught by Jesús to include the directory containing the source 
# within the notebook scope
import sys
sys.path.append('../')

In [21]:
import networkx as nx
import spacy 
from spacy.lang.es.examples import sentences

MODEL = 'es_core_news_sm'
MODEL = 'es_core_news_lg'

# MODEL = 'en_core_web_md'
# MODEL = 'es_core_news_lg'

# we centralize the extraction
nlp = spacy.load(MODEL)
ruler = nlp.add_pipe("entity_ruler")
patterns = [ {"label": "DNI", "pattern": [{"TEXT": {"REGEX": ".*?[0-9]{8}[A-Z]"}}]},
             {"label": "NUM", "pattern": [{"IS_DIGIT":True}]} ]
ruler.add_patterns(patterns)

def generate_textual_patterns_with_pos_tags(corpus, extend_children=False):
    """A method to generate textual patterns given the corpus.

    Parameters
    ----------
    corpus : type List
        List of sentences is passed.

    Returns
    -------
    type List
        List of textual patterns

    """

    patterns_pos_tags = []
    textual_patterns = []
    for i, sentence in enumerate(corpus):
        dep_parse = nlp(sentence)
        # print (f'{i}::{sentence}')
        # print (f'{i}:: Ents{dep_parse.ents}')
        try:
            if len(dep_parse.ents) == 2:
                path = shortest_dependency_path(dep_parse, dep_parse[dep_parse.ents[0].start], dep_parse[dep_parse.ents[1].start])
                # print(f'path: {path}')
                # print(f'dep_parse[[0].start]: {dep_parse[dep_parse.ents[0].start]}')
                # print(f'dep_parse[[1].start]: {dep_parse[dep_parse.ents[1].start]}')
                if len(path) != 2:
                    if (extend_children):
                        path = add_children_deps(path, dep_parse)
                    else:
                        path = [int (x) for x in path]
                    # print (f'path: {path}')
                    ## we add the entity and its POS in parallel
                    shortest_path = dep_parse.ents[0].label_+'_<'+str(dep_parse[dep_parse.ents[0].start:dep_parse.ents[0].end]) + '> '
                    pos_tags = ['<'+dep_parse.ents[0].label_+'>']
                    ## we add all the words in the middle in the same order
                    shortest_path += ' '.join([dep_parse[j].text for j in path[1:-1]])
                    for j in path[1:-1]:
                        pos_tags.append(dep_parse[j].pos_)
                    ## and now the last entity
                    shortest_path += ' '+dep_parse.ents[1].label_+'_<'+str(dep_parse[dep_parse.ents[1].start:dep_parse.ents[1].end]) + '> '
                    pos_tags.append('<'+dep_parse.ents[1].label_+'>')
                    # TODO: update the way of extending with advmod (not yet)
                    # textual_patterns.append(adv_mod_deps(shortest_path, dep_parse))
                    textual_patterns.append(shortest_path)
                    patterns_pos_tags.append(pos_tags)
            elif len(dep_parse.ents)> 2:
                pairs = it.combinations(dep_parse.ents, 2)
                for pair in pairs:
                    # print (f'pair: {pair}')
                    # print(f'dep_parse[pair[0].start]: {dep_parse[pair[0].start]}')
                    # print(f'dep_parse[pair[1].start]: {dep_parse[pair[1].start]}')
                    path = shortest_dependency_path(dep_parse, dep_parse[pair[0].start], dep_parse[pair[1].start])
                    if len(path) != 2:
                        if (extend_children):
                            path = add_children_deps(path, dep_parse)
                        else:
                            path = [int(x) for x in path]
                        # print(f'path: {path}')
                        shortest_path = pair[0].label_+'_<'+ str(dep_parse[pair[0].start:pair[0].end]) + '> '
                        pos_tags = ['<' + pair[0].label_ + '>']
                        shortest_path += ' '.join([dep_parse[j].text for j in path[1:-1]])
                        for j in path[1:-1]:
                            pos_tags.append(dep_parse[j].pos_)
                        shortest_path += ' '+pair[1].label_+'_<'+str(dep_parse[pair[1].start:pair[1].end])+'> '
                        pos_tags.append('<' + pair[1].label_ + '>')
                        # TODO: update this
                        # textual_patterns.append(adv_mod_deps(shortest_path, dep_parse))
                        textual_patterns.append(shortest_path)
                        patterns_pos_tags.append(pos_tags)
        except Exception as e:
            print (e)
            pass
    return textual_patterns, patterns_pos_tags

def add_children_deps(path, dep_parse):
    children_start = obtain_children(int(path[0]), dep_parse)
    children_end = obtain_children(int(path[-1]), dep_parse)
    added_children = [x for x in (children_start + children_end) if int(path[0]) <= int(x) and int(x) <= int(path[-1]) and not is_idx_part_of_entity(int(x), dep_parse)]
    test = [int(x) for x in (path + added_children)]
    if int(path[0]) < int(path[-1]):
        return sorted([int(x) for x in test])
    else:
        return sorted([int(x) for x in test],reverse=True)

def shortest_dependency_path(doc, e1=None, e2=None):
    ## CB: This should also work for NERs, if the dep parsing is pointing at the
    ## head of the entity,
    ## However, the NE would not be added completely
    edges = []
    for token in doc:
        for child in token.children:
            edges.append(('{0}'.format(token.i),
                          '{0}'.format(child.i)))
            edges.append(('{0}'.format(child.i),
                          '{0}'.format(token.i)))

    graph = nx.Graph(edges)
    shortest_path = []
    try:
        # print (f'looking shortest path from {e1.i} to {e2.i}')
        shortest_path = nx.shortest_path(graph, source=str(e1.i), target=str(e2.i))
    except nx.NetworkXNoPath:
        print (f'problems with shortest_path - {str(e1.i)} {doc[e1.i]} - {str(e2.i)} {doc[e2.i]}')
        print (graph)
        shortest_path=[]
    return shortest_path



In [22]:
sents = []
for s in sentences:
    sents.append(s)

textual_patterns, post = generate_textual_patterns_with_pos_tags(sents, True)
print (textual_patterns)
print (f'Done. ')

[WinError 126] No se puede encontrar el módulo especificado
[WinError 126] No se puede encontrar el módulo especificado
[WinError 126] No se puede encontrar el módulo especificado
[WinError 126] No se puede encontrar el módulo especificado
[]
Done. 
